## 1.引言

对模型的微调任务一般分为两类：分类微调和指令微调。我们在[上一节](https://golfxiao.blog.csdn.net/article/details/144910832) 介绍了分类微调的训练过程，从这一节开始将进入指令微调。

前面的预训练让模型学会了如何在每次运算中生成下一个token，因此预训练模型很擅长文本补全，但它不太擅长遵循人类的指令，而指令微调的目的就是让模型学会如何遵循人类的指令。

![指令遵循](./img/12-1.jpg)

指令微调相比于预训练来说，最大的区别在于数据部分。由于指令微调的目的是让模型学会如何与人类对话，所以指令微调的数据内容会更复杂，所需要做的数据处理工作也会更多，本节我们将重点介绍如何为指令微调准备数据。

## 2. 准备数据

#### 2.1 获取数据集

先下载指令微调数据集。
```
!curl -o /data2/minigpt/dataset/sft/sft_data_zh.jsonl "https://www.modelscope.cn/datasets/deepctrl/deepctrl-sft-data/resolve/master/sft_data_zh.jsonl"
```

我们这里所选择的数据集是一个jsonl格式的文件，大小17GB，如下所示。

In [2]:
!ls -n /data2/minigpt/dataset/sft/sft_data_zh.jsonl

-rw-rw-r-- 1 1010 1010 17193006547 Sep 26 22:51 /data2/minigpt/dataset/sft/sft_data_zh.jsonl


对于如此大的数据集文件，常规机器是无法一次性读进内存的，这时我们需要使用到一个模块`linecache`。

linecache是python的一个内置模块，可以支持按行读取文件，并利用内存来缓存读取到的内容，以支持后续快速访问。

In [6]:
import linecache

data_path = "/data2/minigpt/dataset/sft/sft_data_zh.jsonl"
line = linecache.getline(data_path, 3329085)
line

'{"id": 3405593, "instruction": "请判断下面文本属于哪种主题类型，类型包括: 星座、房产、股票、教育、社会、财经、游戏、家居、娱乐、时尚、时政、体育、彩票、科技。\\n请直接输出类型名称，不要额外输出多余内容。\\n", "input": "刘鹏，体育彩票是腐败高危区域，我们坐在火山口上\\n", "output": "彩票", "history": [], "language": "chinese", "data_source": "https://huggingface.co/datasets/BAAI/COIG-PC-core", "input_len": 111, "output_len": 2, "num_utter": 1, "type": 12, "type_keyword": ["问题", "文本", "是否", "进行", "中国", "公司", "没有", "实体", "情感", "主题"]}\n'

可以看到，这个数据集中一条数据的字段有些多，我们可以选择性的忽略其它字段，只关注`instruction`、`input`、`history`、`output`四个字段，分别表示指令、输入、历史消息、目标输出，这也是指令微调数据组成中最基本的几项信息。

由于指令微调是在带有明确`input-output`对的数据集上进行训练，因此指令微调也被称为监督指令微调。

#### 2.2 数据格式化
大模型是支持与人对话聊天的，每个聊天有一到多条消息组成，每条消息都有一个角色和内容，就像ChatGPT一样。因此，我们自己定义的模型MiniGPT也需要一个输入模板。

每个模型通常都有自己固定的输入模板格式，这个输入模板是可以被学习的，模型通常会在训练阶段学习这个格式，目的是学会一段指令或文本在什么地方开始，又在什么地方结束，这就像我们人类读一篇课文时，看到句号`。`就知道是一句话结束了。

我们在前面的[分词器训练]()一节中，给分词器编写配置文件时曾经配置过chat_template，下面我们将分词器加载进来，看下输入定义的输入模板格式。

In [10]:
from transformers  import AutoTokenizer

tokenizer_path = "/data2/minigpt/models/tokenizer_v3"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, use_fast=True)
tokenizer.chat_template

"{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ system_message }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|im_start|>user\\n' + content + '<|im_end|>\\n<|im_start|>assistant\\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|im_end|>' + '\\n' }}{% endif %}{% endfor %}"

这个输入模板中共支持三个角色的消息，分别是`system`、`user`、`assistant`，但是不够直观。下面我们尝试运用这个模板对上面的单条输入`line`数据进行格式化。

In [11]:
import json 

def apply(data_item, tokenize=False):
    messages = [
        {"role": "user", "content": f"{data_item['instruction']}\n{data_item['input']}"},
        {"role": "assistant", "content": data_item["output"]},
    ]
    return tokenizer.apply_chat_template(messages, tokenize=tokenize, add_generation_prompt=True)

data_item = json.loads(line)
apply(data_item)

'<|im_start|>user\n请判断下面文本属于哪种主题类型，类型包括: 星座、房产、股票、教育、社会、财经、游戏、家居、娱乐、时尚、时政、体育、彩票、科技。\n请直接输出类型名称，不要额外输出多余内容。\n\n刘鹏，体育彩票是腐败高危区域，我们坐在火山口上\n<|im_end|>\n<|im_start|>assistant\n彩票<|im_end|>\n'

上面`apply`方法中tokenize参数的作用是：是否在格式化的同时对输入消息由文本序列化为token ID列表。如果我们传True，上面apply函数将输出如下结果。

In [7]:
print(apply(data[0], tokenize=True))

[1, 17681, 201, 39, 88, 3739, 429, 276, 3428, 7404, 645, 18203, 470, 1322, 276, 14808, 4755, 16, 201, 17250, 807, 25084, 2487, 2, 201, 1, 1078, 9368, 201, 1148, 14808, 303, 276, 4755, 7404, 764, 17250, 807, 4, 370, 16661, 14, 276, 5190, 14808, 370, 764, 4338, 6492, 2, 201]


#### 2.3 自定义数据集类

这一部分的目标是构建一个适用于指令微调场景的自定义数据集类`SFTDataset`

第一步，导入必要的库，其中：
- torch.utils.data.Dataset 是 PyTorch 提供的一个数据集基类，我们将基于这个基类来构建自己的数据集类。
- json用于解析jsonl格式的数据；
- linecache用于高效读取文件中的特定行。

In [19]:
from torch.utils.data import Dataset
import json
import time
import linecache

In [ ]:
第二步，定义数据集类，并初始化关键信息，包括：
- 分词器tokenizer; 
- 序列最大长度max_len，默认为1024; 
- jsonl格式的数据集文件路径； 
- 数据集最大条数max_lines，用于只想使用一部分数据进行训练的场景。

In [41]:
class InstructionDataset(Dataset):
    def __init__(self, jsonl_file_path, tokenizer, max_len=1024, max_lines=0):
        self.jsonl_file_path = jsonl_file_path
        self.tokenizer = tokenizer
        self.max_len = max_len
        start_time = time.time()
        # 如果入参没有指定最大条数，则自动计算文件中的总行数作为总条数，反之则以入参指定的总条数为准
        if max_lines <= 0:
            with open(self.jsonl_file_path, 'r', encoding='utf-8') as f:
                self.total_lines = sum(1 for _ in f)
        else:
            self.total_lines = max_lines
        print(f"calculate lines[{self.total_lines}] use time: {time.time()-start_time:.3f}s")

> 注：关于上面计算数据总条数的代码，`sum(1 for _ in f)`就是对文件中出现的每一行都按1计数，最后用sum函数求和就可得到文件的总行数，这种计算方法只适用于jsonl格式（一行一条数据）的数据集。

第三步，实现标准的`__len__`方法用于返回数据集中的总数据条数。

In [43]:
def __len__(self):
    return self.total_lines

setattr(InstructionDataset, "__len__", __len__)

第四步，自定义一个数据处理的方法`process`，用于格式化数据。

我们会在上面`apply`函数的基础之上添加对历史消息`history`的支持，并且要求history中的消息应该是`user`和`assistant`两个角色的发言成对出现的。

In [44]:
def process(self, item):
    messages = []
    for history_item in item.get("history", []):
        if len(history_item) < 2:
            continue
        messages.append({"role": "user", "content": history_item[0][:self.max_len//2]})
        messages.append({"role": "assistant", "content": history_item[1][:self.max_len//2]})
    
    user_content = item['instruction'] + '\n' + item['input']
    assistant_content = item['output']
    messages.append({"role": "user", "content": user_content})
    messages.append({"role": "assistant", "content": assistant_content})
    input_ids =  self.tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=False)
    return input_ids[:self.max_len]

setattr(InstructionDataset, "process", process)

最后一步，实现标准的读取数据方法__getitem__, 根据索引下标来读取数据。

此方法会先从linecache中读取指定行的原始json数据，经过解析后调用上面定义的process方法来处理数据，最后返回序列化后的数据。并且此方法对单条读和多个索引批量读均作了支持。

In [45]:
def __getitem__(self, idx):
    indexes = [idx] if isinstance(idx, int) else idx
    assert isinstance(indexes, list)

    # 使用 linecache 读取指定行
    lines = [linecache.getline(self.jsonl_file_path, i + 1).strip() for i in indexes]
    datas = [json.loads(line) for line in lines]
    inputs =  [self.process(data) for data in datas]
    return inputs[0] if isinstance(idx, int) else inputs

setattr(InstructionDataset, "__getitem__", __getitem__)

我们用本节开头下载过的数据集文件`sft_data_zh.jsonl`来测试下刚封装的`SFTDataset`。

In [46]:
data_path = "/data2/minigpt/dataset/sft/sft_data_zh.jsonl"
mini_ds = InstructionDataset(data_path, tokenizer, max_lines=1024)
len(mini_ds)

calculate lines[1024] use time: 0.000s


1024

访问数据集中的第一条数据。

In [47]:
print(mini_ds[0])

[1, 17681, 201, 5715, 28162, 315, 7878, 3147, 270, 9480, 9690, 3373, 2451, 5715, 7878, 269, 644, 3966, 286, 201, 1966, 14084, 6593, 3560, 3036, 270, 12262, 9806, 2933, 31843, 14626, 1083, 990, 12629, 9480, 7878, 10508, 62, 80, 7878, 3147, 17122, 25380, 3560, 1860, 270, 25380, 2933, 1860, 63, 2, 201, 1, 1078, 9368, 201, 9480, 3373, 2451, 5715, 7878, 269, 644, 3966, 799, 270, 1966, 17530, 14084, 6593, 3036, 270, 12262, 9806, 31843, 14626, 1083, 990, 12629, 9480, 7878, 10508, 2, 201, 1, 17681, 201, 201, 1188, 286, 9823, 21151, 9690, 7078, 269, 25327, 959, 11049, 518, 15954, 286, 2, 201, 1, 1078, 9368, 201, 1188, 270, 1055, 28666, 2336, 270, 2525, 4659, 658, 9690, 7078, 25327, 24568, 15954, 286, 1958, 799, 1966, 428, 14084, 6593, 3036, 223, 12262, 9806, 31843, 14626, 1083, 990, 12629, 9480, 7878, 10508, 286, 1958, 2103, 1422, 814, 2, 201]


> 注：上面访问`mini_ds[0]`就等价于调用InstructionDataset类的`__getitem__`方法，它自动对明文的jsonl格式数据进行处理，并序列化成了token ID列表。

我们可以用tokenizer的反序列化来查看下`mini_ds[0]`对应的明文内容。

In [39]:
tokenizer.decode(mini_ds[0])

'<|im_start|>user\n给定一段文本和关键词列表，删除文本中包含所有给定关键词的子字符串。\n文本："这是一个测试句子，目的是看看模型是否可以正确地从这个句子中删除关键词。"\\n关键词列表：[‘测试’，‘模型’]<|im_end|>\n<|im_start|>assistant\n删除包含所有给定关键词的子字符串后，文本变为："这是一个句子，目的是看看是否可以正确地从这个句子中删除关键词。"<|im_end|>\n<|im_start|>user\n\n好的。现在请你将这个文本中的所有的逗号都替换成空格。<|im_end|>\n<|im_start|>assistant\n好的，请稍等一下，现在我会将文本中的所有逗号替换为空格。处理后文本为："这是一个句子 目的是看看是否可以正确地从这个句子中删除关键词。"。处理结果如何？<|im_end|>\n'

## 3. 批量数据

#### 3.1 输入填充
同上一节的分类微调相似，我们在使用小批量梯度下降方法来训练模型时，要求一个批量内的数据都具有相同的长度。在上一节的SpamDataset中我们简单的将整个数据集都补齐到同一个长度，这一节我们会采用一种更精细化的方法，引入一个专用于批量数据制作的collate函数，此函数可以传递给dataloader，在进行批量迭代时将每个批量补齐为相同的长度，但不同的批量可以有不同的长度。

![不同批量不同长度](./img/12-2.jpg)

如同上一节一样，使用`<|endoftext|>`作为padding token。

In [49]:
pad_token, pad_token_id = tokenizer.unk_token, tokenizer.unk_token_id
pad_token, pad_token_id

('<|endoftext|>', 0)

我们来实现一个collate函数，对输入的一个批量数据作填充（如下）。

In [ ]:
import torch 

def collate(batch, pad_token_id, device='cpu'):
    assert isinstance(pad_token_id, int)
    max_length = max([len(item) for item in batch])
    # pad each sequence to max_length
    padded_batch = [item + [pad_token_id] * (max_length - len(item)) for item in batch]
    return torch.tensor(padded_batch, dtype=torch.int64).to(device)


考虑到正式数据集中的数据都比较长，这里我们定义一个比较简单的样例数据`example_data`，批量为2且指令、输入、输出都比较短，这样就能控制下序列长度方便演示，并用此数据来测试下collate函数。

In [54]:
example_data = [
  {
    "instruction": "将以下句子翻译成英文: ",
    "input": "'我喜欢编程'",
    "output": "I like programming",
  },
  {
    "instruction": "计算 25 加 17",
    "input": "",
    "output": "42",
  }
]

batch_inputs = [apply(item, True) for item in example_data]
batch_padded = collate(batch_inputs, pad_token_id)
batch_padded

tensor([[    1, 17681,   201, 25185, 24937,    28,   223,   201,     9, 12218,
          4399,     9,     2,   201,     1,  1078,  9368,   201,    43,  1011,
          5788,     2,   201],
        [    1, 17681,   201,  1429,  6464, 20053,  8989,   201,     2,   201,
             1,  1078,  9368,   201, 14263,     2,   201,     0,     0,     0,
             0,     0,     0]])

> 上面第一条数据比第二条数据长，所以经过collate函数处理后，第二条数据用0(pad_token_id)作了填充，长度补齐到和第一条一致。

在这一步中，我们只是返回了输入数据, 但在LLM的训练中，我们还需要返回目标数据labels和注意力掩码attention_mask。

#### 3.2 注意力掩码
先来计算注意力掩码，它的目的在于对输入内容计算注意力时忽略掉没有意义的内容，也就是padding部分。计算方法是为每一个输入序列创建一个与序列等长的掩码序列，掩码序列中只有两个值0和1，0表示要忽略的padding部分，1表示要关注的非padding部分。

In [92]:
input_ids = batch_padded[1]

attention_mask = torch.ones(len(input_ids), dtype=torch.int64)
attention_mask.masked_fill(input_ids == pad_token_id, 0)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0])

#### 3.3 目标数据

对于目标数据，我们需要做两个处理：
1. 将输入数据中的每个token沿着下标向右移动一个位置，目标是让模型学习预测下一个token。
2. 需要引入损失掩码，目的是对指令和输入部分不计算损失。


###### 生成目标序列
首先，移动输入的token序列得到目标token序列。

In [57]:
old_input_ids = batch_padded[1].tolist()
input_item = old_input_ids + [pad_token_id]
input_ids = input_item[:-1]
target_ids = input_item[1:]
print("input_ids: ", input_ids)
print("target_ids:", target_ids)
print("new_input == old_input: ", input_ids == old_input_ids)

input_ids:  [1, 17681, 201, 1429, 6464, 20053, 8989, 201, 2, 201, 1, 1078, 9368, 201, 14263, 2, 201, 0, 0, 0, 0, 0, 0]
target_ids: [17681, 201, 1429, 6464, 20053, 8989, 201, 2, 201, 1, 1078, 9368, 201, 14263, 2, 201, 0, 0, 0, 0, 0, 0, 0]
new_input == old_input:  True


> 我们先对input扩展了一个pad_token，这样在向右移动一个位置时就不会出界。从上面的输出可以看到，target_ids与input_ids正好错了一个位置。

###### 引入损失掩码
接下来，需要区分出指令输入和输出部分，apply函数对提示语标准化后的结果示例如下：

In [55]:
print("input[0]:", repr(apply(example_data[0])))
print("input[1]:", repr(apply(example_data[1])))

input[0]: "<|im_start|>user\n将以下句子翻译成英文: \n'我喜欢编程'<|im_end|>\n<|im_start|>assistant\nI like programming<|im_end|>\n"
input[1]: '<|im_start|>user\n计算 25 加 17\n<|im_end|>\n<|im_start|>assistant\n42<|im_end|>\n'


从上面标准化后的结果示例可以看出，输出部分是从`<|im_start|>assistant\n`这个子串开始的。我们可以尝试找出这个子串的位置，进而区分出指令输入和输出。

In [59]:
def find_sublist_index(main_list, sub_list):
    for i in range(len(main_list) - len(sub_list) + 1):
        if main_list[i:i + len(sub_list)] == sub_list:
            return i
    return -1

seperator = tokenizer("<|im_start|>assistant\n")['input_ids']
output_start_index = find_sublist_index(target_ids, seperator)
instruction_length = output_start_index + len(seperator)
instruction_ids, output_ids = target_ids[:instruction_length], target_ids[instruction_length:]
print("seperator:", seperator)
print("instruction_ids:", instruction_ids)
print("output_ids:", output_ids)

seperator: [1, 1078, 9368, 201]
instruction_ids: [17681, 201, 1429, 6464, 20053, 8989, 201, 2, 201, 1, 1078, 9368, 201]
output_ids: [14263, 2, 201, 0, 0, 0, 0, 0, 0, 0]


输出序列target_ids已经从输出标记`[1, 1078, 9368, 201]`的位置分割成了指令输入和模型输出两部分。

虽然整个target_ids模型都会进行预测，但只有output_ids中的`[14263, 2, 201]`才是我们真正期望模型输出的内容，并且也只需要对这一部分计算损失。所以我们需要找一个机制来告诉模型有两部分内容不需要计算损失：instruction_ids和padding。而这个机制是通过引入一个损失掩码-100来遮盖不需要计算损失的部分。

In [110]:
ignore_token_id = -100
batch_targets = [-100 if (item == pad_token_id or i < instruction_length) else item for i, item in enumerate(target_ids)]
print("labels.length == target_ids.length: ", len(batch_targets) == len(target_ids))
print(batch_targets)

labels.length == target_ids.length:  True
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 14263, 2, 201, -100, -100, -100, -100, -100, -100, -100]


通过上面的运算，已经将目标输出中的指令输入和padding都替换成了损失掩码-100。

这里有个疑问：这个-100是怎么来的，是否真的起作用？我们可以尝试验证下。

###### 损失掩码功能验证

In [111]:
logits_1 = torch.tensor([
    [-1.0, 1.0],
    [-0.5, 1.5],
])
targets_1 = torch.tensor([0, 1])

loss1 = torch.nn.functional.cross_entropy(logits_1, targets_1)
loss1

tensor(1.1269)

In [118]:
logits_2 = torch.tensor([
    [-1.0, 1.0],
    [-0.5, 1.5],
    [-3.0, 1.3],
])
targets_2 = torch.tensor([0, 1, 0])
loss2 = torch.nn.functional.cross_entropy(logits_2, targets_2)
loss2

tensor(2.1891)

上面两个损失的计算中，loss2只是在loss1的基础上多了第三个元素的损失计算，如果我们将第三个元素的目标输出设为-100，看看会发生什么。

In [119]:
targets_3 = torch.tensor([0, 1, -100])
loss3 = torch.nn.functional.cross_entropy(logits_2, targets_3)
loss3, loss3 == loss1

(tensor(1.1269), tensor(True))

可以看到，第三个损失的计算结果与第一个完全相等，证明损失掩码-100确实能起到在计算损失时遮蔽指定位置的效果。

> 实际上，pytorch的交叉熵函数`cross_entropy(..., ignore_index=-100)`是有一个参数专门用来指定损失掩码数值的，它默认值被设为了-100。

#### 3.4 最终collate函数

上面整个批量数据的处理过程可以整理为两个函数：calc_label和collate，其中`calc_label`用于计算目标序列，collate则用于得到最终的小批量输入序列、掩码序列和输出序列。

In [68]:
def calc_label(input_ids, pad_token_id, tokenizer):
    target_ids = input_ids[1:] + [pad_token_id]
    output_seperator = tokenizer("<|im_start|>assistant\n")['input_ids']
    output_start_index = find_sublist_index(target_ids, output_seperator)
    instruction_length = output_start_index + len(output_seperator)
    label_ids = [-100 if (item == pad_token_id or i < instruction_length) else item for i, item in enumerate(target_ids)]
    return label_ids


In [69]:
import numpy as np 

def collate(batch_inputs, pad_token_id, tokenizer, device='cpu'):
    assert isinstance(pad_token_id, int)
    # pad each sequence to max_length
    max_length = max([len(item) for item in batch_inputs])
    batch_padded = [item + [pad_token_id] * (max_length - len(item)) for item in batch_inputs]
    input_tensors = torch.tensor(batch_padded, dtype=torch.int64).to(device)

    attention_mask = torch.ones(input_tensors.shape, dtype=torch.int64).to(device)
    attention_mask = attention_mask.masked_fill(input_tensors == pad_token_id, 0)

    batch_targets = [calc_label(item, pad_token_id, tokenizer) for item in batch_padded]
    target_tensors = torch.tensor(batch_targets, dtype=torch.int64).to(device)
    return input_tensors, attention_mask, target_tensors

测试新封装的collate批量处理函数效果。

In [65]:
collate(batch_inputs, pad_token_id, tokenizer)

(tensor([[    1, 17681,   201, 25185, 24937,    28,   223,   201,     9, 12218,
           4399,     9,     2,   201,     1,  1078,  9368,   201,    43,  1011,
           5788,     2,   201],
         [    1, 17681,   201,  1429,  6464, 20053,  8989,   201,     2,   201,
              1,  1078,  9368,   201, 14263,     2,   201,     0,     0,     0,
              0,     0,     0]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]),
 tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,    43,  1011,  5788,
              2,   201,  -100],
         [ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100, 14263,     2,   201,  -100,  -100,  -100,  -100,
           -100,  -100,  -100]]))

> 上面输出的3个tensor分别为输入batch、注意力掩码、输出batch。

## 4. Dataloader
这一部分我们将会综合上面的内容，来制作一个适用于指令微调场景的小批量数据加载器。

首先，我们加载正式的数据集。

In [60]:
data_path = "/data2/minigpt/dataset/sft/sft_data_zh.jsonl"
ds = InstructionDataset(data_path, tokenizer)
len(ds)

calculate lines[11381621] use time: 52.639s


11381621

将数据集以`8:1:1`的比例分割成训练集、验证集和测试集。

In [63]:
def split_dataset(data, train_ratio, eval_ratio):
    train_len = int(len(data) * train_ratio)
    eval_len = int(len(data) * eval_ratio)
    test_len = len(data) - train_len - eval_len
    return torch.utils.data.random_split(data, [train_len, eval_len, test_len])

train_set, eval_set, test_set = split_dataset(ds, 0.8, 0.1)
len(train_set), len(eval_set), len(test_set)

(9105296, 1138162, 1138163)

由于collate函数需要一些参数，并不方便传递，因此，使用partial语法来将参数固定住，对外暴露一个无参新函数`batch_collator`。

In [79]:
from functools import partial

def create_batch_collator(tokenizer):
    return partial(
        collate,
        pad_token_id = tokenizer.unk_token_id,
        tokenizer = tokenizer, 
    )

batch_collator = create_batch_collator(tokenizer)

基于上面分割的数据集来创建批量数据加载器，与之前唯一的区别是我们指定了自己的批量数据处理函数`collate_fn`。

In [80]:
from torch.utils.data import DataLoader

batch_size = 8
num_workers = 0

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True, collate_fn=batch_collator)
eval_loader = DataLoader(eval_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=False, collate_fn=batch_collator)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=False, collate_fn=batch_collator)

for i in range(2): 
    inputs, attention_mask, targets = next(iter(train_loader))
    print("batch:", i+1, ", inputs.shape: ", inputs.shape, ", attention_mask.shape: ", attention_mask.shape, ", targets.shape: ", targets.shape)

batch: 1 , inputs.shape:  torch.Size([8, 541]) , attention_mask.shape:  torch.Size([8, 541]) , targets.shape:  torch.Size([8, 541])
batch: 2 , inputs.shape:  torch.Size([8, 1024]) , attention_mask.shape:  torch.Size([8, 1024]) , targets.shape:  torch.Size([8, 1024])


> 正如上面输出中所示，每个批量的size都是8个序列，但各个批量的序列长度却不同，并且每个批量内部各个序列的输入和输出形状是完全相同的。

**小结**：本节我们主要介绍了指令微调所需要的数据构造过程，相比于预训练来讲指令微调的数据构造过程要更复杂，主要在于以下几点：
- **格式**：为了学习与人类聊天对话，指令微调的训练数据引入了聊天模板`chat_template`，引入了不同角色`user`、`assistant`和`system`；
- **注意力掩码**：由于填充的原因，指令微调引入了注意力掩码来忽略填充token，而预训练中不存在填充，因为所有数据的长度都是context_length；
- **损失掩码**：指令微调引入了损失掩码的概念，因为指令微调生成序列中只有目标输出`output`是我们关心的内容，前面的instruction和input部分我们并不关心，因此用损失掩码将其屏蔽。而在预训练阶段模型生成的整个序列我们都关心（不需要损失掩码）。